注意该文件的运行环境为kaggle

In [ ]:
!pip install evaluate
import os
import sys
import logging
import datasets
import evaluate

import pandas as pd
import numpy as np

from transformers import AutoModelForSequenceClassification, AutoTokenizer, DataCollatorWithPadding
from transformers import Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model, TaskType
from sklearn.model_selection import train_test_split

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 8.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.9.0.13 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudnn

2025-05-13 15:50:33.883552: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747151434.072178      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747151434.124837      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


##  数据预处理

In [2]:
trainDataPath = r"/kaggle/input/imdb-sentiment-analysis/labeledTrainData.tsv"
testDataPath = r"/kaggle/input/imdb-sentiment-analysis/testData.tsv"

train = pd.read_csv(trainDataPath, header=0, delimiter="\t", quoting=3)
test = pd.read_csv(testDataPath, header=0, delimiter="\t", quoting=3)

print(train.shape)

(25000, 3)


In [4]:
train, val = train_test_split(train, test_size=0.2)
print(train.shape, val.shape)

train_dict = {'label': train["sentiment"], 'text': train['review']}
val_dict = {'label': val["sentiment"], 'text': val['review']}
test_dict = {"text": test['review']}

train_dataset = datasets.Dataset.from_dict(train_dict)
val_dataset = datasets.Dataset.from_dict(val_dict)
test_dataset = datasets.Dataset.from_dict(test_dict)
print(train_dataset)

(16000, 3) (4000, 3)
Dataset({
    features: ['label', 'text'],
    num_rows: 16000
})


In [ ]:
checkpoint = "microsoft/deberta-v2-xxlarge"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example["text"], truncation=True, max_length=512)

tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_val = val_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)
print(tokenized_train)

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.45M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/633 [00:00<?, ?B/s]

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Dataset({
    features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 16000
})


## 创建模型

In [6]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Define LoRA Config
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    task_type=TaskType.SEQ_CLS
)

model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

pytorch_model.bin:   0%|          | 0.00/3.14G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.14G [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v2-xxlarge and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 4,721,666 || all params: 1,571,635,204 || trainable%: 0.3004


## Trainer

In [7]:
def compute_metrics(eval_preds):
    metric = evaluate.load("accuracy")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [8]:
training_args = TrainingArguments(
        run_name="deberta-lora",
        output_dir='./deberta-v2-xxlarge-lora',  
        learning_rate=1e-5,
        num_train_epochs=3,  
        per_device_train_batch_size=1,  
        per_device_eval_batch_size=1,  
        weight_decay=0.01,
        eval_strategy="epoch",
        report_to="none",
)

In [ ]:
trainer = Trainer(
    model=model,  
    args=training_args,  
    train_dataset=tokenized_train, 
    eval_dataset=tokenized_val, 
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.234300,0.236316,0.962250
2,0.131400,0.210252,0.964000
3,0.321800,0.209702,0.964750


TrainOutput(global_step=48000, training_loss=0.2467529279390971, metrics={'train_runtime': 25713.544, 'train_samples_per_second': 1.867, 'train_steps_per_second': 1.867, 'total_flos': 1.0551498871195813e+17, 'train_loss': 0.2467529279390971, 'epoch': 3.0})

## 预测，并保存数据

In [10]:
import os

# 确保result文件夹存在
os.makedirs("/kaggle/working/result", exist_ok=True)  # 自动创建且不报错（Python 3.2+）

prediction_outputs = trainer.predict(tokenized_test)
test_pred = np.argmax(prediction_outputs[0], axis=-1).flatten()
print(test_pred)

result_output = pd.DataFrame(data={"id": test["id"], "sentiment": test_pred})
result_output.to_csv("/kaggle/working/result/deberta_lora.csv", index=False, quoting=3)

[1 0 0 ... 0 1 1]
